In [ ]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 이미지 분류 연습 - 머신러닝(KNN)

K-NN 알고리즘을 사용하여 이미지 분류하기

K-NN 알고리즘은 무엇인가?

거리 기반 알고리즘

In [2]:
import pandas as pd
import numpy as np

from skimage.transform import resize
from skimage.io import imread





import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] =False

import os



In [ ]:
datadir = '/content/drive/MyDrive/project3/data/traindata/kfood2'

food_middle_list = os.listdir(datadir)
food_middle_list = sorted(food_middle_list)
food_middle_list[:5]

['구이', '국', '기타', '김치', '나물']

In [ ]:
food_list = [] # 음식이름
food_path_list = [] # 음식이름 경로

for food_middle in food_middle_list:
    middle_path = os.path.join(datadir,food_middle)
    food_name_list = os.listdir(middle_path)
    for food in food_name_list:
        path = os.path.join(middle_path,food)
        food_path_list.append(path)

        name = path.split('/')[-1]
        food_list.append(name)

print(len(food_list))
print(food_list)
print(food_path_list)

150
['갈비구이', '갈치구이', '고등어구이', '곱창구이', '장어구이', '떡갈비', '더덕구이', '불고기', '조개구이', '삼겹살', '조기구이', '닭갈비', '훈제오리', '황태구이', '계란국', '떡국_만두국', '무국', '미역국', '시래기국', '콩나물국', '육개장', '북엇국', '젓갈', '양념치킨', '과메기', '콩자반', '편육', '피자', '후라이드치킨', '갓김치', '깍두기', '나박김치', '무생채', '배추김치', '백김치', '부추김치', '열무김치', '오이소박이', '파김치', '총각김치', '가지볶음', '시금치나물', '애호박볶음', '고사리나물', '숙주나물', '미역줄기볶음', '경단', '꿀떡', '송편', '만두', '라면', '막국수', '물냉면', '비빔냉면', '수제비', '열무국수', '콩국수', '짬뽕', '잔치국수', '칼국수', '짜장면', '쫄면', '꽈리고추무침', '도라지무침', '도토리묵', '잡채', '콩나물무침', '회무침', '홍어무침', '누룽지', '김밥', '김치볶음밥', '비빔밥', '새우볶음밥', '알밥', '유부초밥', '잡곡밥', '주먹밥'

In [ ]:
target = []
images = []   # In matrix format
flat_data = []  # In vector format

datadir = '/content/drive/MyDrive/project3/data/traindata/kfood2'
# food_path_list = food_path_list[:5]

for idx, food_path in enumerate(food_path_list):
    food = food_path.split('/')[-1]

    for img in os.listdir(food_path):
        img_path = os.path.join(food_path,img)
        img_type = img_path.split('.')[-1]
        if (img_type != 'properties') & (img_type != 'csv'):
            img_matrix = imread(img_path)
            img_resized = resize(img_matrix,(150,150,3))
            flat_data.append(img_resized.flatten())
            images.append(img_resized)
            target.append(food)

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
target = []
images = []   # In matrix format
flat_data = []  # In vector format

datadir = '/content/drive/MyDrive/project3/data/traindata/kfood2'
# food_path_list = food_path_list[:5]

for idx, food_path in enumerate(food_path_list):
    food = food_path.split('/')[-1]

    for img in os.listdir(food_path):
        img_path = os.path.join(food_path,img)
        flat_data.append(img_path)


In [ ]:
len(flat_data)

140086

In [ ]:
import pandas as pd
df=pd.DataFrame(flat_data)
df['Target']=target

In [ ]:
df.shape

In [ ]:
# Display of the number of images of each Food type
plt.bar(df['Target'].unique(),df['Target'].value_counts())
plt.title('Food vs Quantity')
plt.xlabel('Food Name')
plt.ylabel('No. of Foods')
plt.show()


In [ ]:
# Input and Output
x=df.iloc[:,0:67500].values
y=df.iloc[:,67500].values

3) Classification of images using KNN Classification Algorithm

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
# k=np.arange(1,160)
k = [i if i != 0 else 1 for i in range(0, 160, 5)]
param={'n_neighbors':k}
model=KNeighborsClassifier()
model_grid=GridSearchCV(model,param)
model_grid.fit(x,y)

In [ ]:
# Best K nearest neighbor
model_grid.best_params_

In [ ]:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
best_k = model_grid.best_params_['n_neighbors']
model = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
model.fit(x_train,y_train)

In [ ]:
# Predicted output
y_pred = model.predict(x_test)
y_pred

In [ ]:
# Actual output
y_test

In [ ]:
# Evaluation of the model
from sklearn.metrics import accuracy_score
print("The accuracy of the model is ",accuracy_score(y_pred,y_test)*100,"%")

# 4) Confusion matrix and Classification report

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred,y_test)
cm

In [ ]:
# Classification report
from sklearn import metrics
metrics.classification_report(y_test,y_pred)

5) Model save

In [ ]:
import joblib

# 모델 저장
joblib.dump(model, '/content/drive/MyDrive/project3/model/image/KNN/knn_model.pkl')

# 모델 불러오기
knn_model = joblib.load('/content/drive/MyDrive/project3/model/image/KNN/knn_model.pkl')

# 5) Prediction for new image

In [ ]:
# # Resizing and flattening new image 1
# img1=imread('/content/drive/My Drive/images/New prediction images/plantsguru-flowering-plants-rose-red-800x800.jpg')
# img1_resize = resize(img1,(150,150,3))
# img1_flatten = img1_resize.flatten()


In [ ]:

# img1_pred = model.predict([img1_flatten])
# img1_pred

In [ ]:

# plt.imshow(img1)